In [39]:
# import os, getpass
from ibm_watsonx_ai import Credentials, APIClient
from langchain_chroma import Chroma
from langchain_text_splitters import MarkdownTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import ChatWatsonx
# from langchain.chains import RetrievalQAjin
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from docling.document_converter import DocumentConverter
from utils import *
PAGE_BREAK = "<!-- page break -->"

In [2]:
api_key="VPryyL4t508YQ_pJQNCU0m23Yctctahrqo2pjT5BMQkV"
url="https://us-south.ml.cloud.ibm.com"
project_id = "6552c3bb-a35a-4bf3-8430-c42828d59a06"

## Load documents

In [3]:
## TODO preserve metadata e.g. page number, chunking should probably be done per page

file_path = "data/climate_edu/389801eng.pdf"
converter = DocumentConverter()
result = converter.convert(file_path)
doc = result.document


/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/docling/pipeline/standard_pdf_pipeline.py:262: RuntimeWarning: Mean of empty slice
  np.nanmean(


In [25]:
doc_md = doc.export_to_markdown(page_break_placeholder=PAGE_BREAK, image_placeholder="",)
with open("data/climate_edu/389801eng.md", "w") as f:
    f.write(doc_md)

pages = doc_md.split(PAGE_BREAK)
pages = [markdown_cleanup(page) for page in pages]


## Chunk

In [40]:
## TODO change model to granite embed
tokenizer = AutoTokenizer.from_pretrained(
    "ibm-granite/granite-embedding-278m-multilingual"
)


markdown_splitter = MarkdownTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=500,
    chunk_overlap=50,
)

In [46]:
chunked_doc = []
for i, page in enumerate(pages):
    chunks = markdown_splitter.split_text(page)
    for j, chunk in enumerate(chunks):
        chunked = {
            "page": i + 1,
            "chunk": j + 1,
            "text": chunk
        }
        chunked_doc.append(chunked)

In [ ]:
# filename = "data/email.csv"
# loader = TextLoader(filename)
# documents = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)
# texts

In [ ]:
model_id = "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"
parameters = TextChatParameters(
    max_tokens = 1024,
    temperature = 0
)
watsonx_llm = ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=parameters,
)

In [ ]:
messages = [
    ("system", "You are a helpful assistant that translates English to French."),
    (
        "human",
        "I love you for listening to Rock.",
    ),
]


In [ ]:

out = watsonx_llm.invoke(messages)


In [ ]:
out.content